# **Card Detection Image Processing with YOLOV8**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from ultralytics import YOLO
from roboflow import Roboflow
from torchvision import transforms
from torch.utils.data import Dataset
import os
from PIL import Image

rf = Roboflow(api_key="AiIAflLM41FNvbBxejqZ") 
project = rf.workspace("khomsan-63wkg").project("bo-amjwf")
download_dir = "content/datasets/"
version = project.version(1)
dataset = version.download("yolov8", download_dir)

model = YOLO('yolov8s.pt')  

ModuleNotFoundError: No module named 'roboflow'

In [ ]:
transform = transforms.Compose([
    transforms.Resize((640, 640)), 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

In [ ]:
class CustomYOLODataset(Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = os.listdir(img_dir)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx].replace(".jpg", ".txt"))
        
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        with open(label_path) as f:
            labels = f.readlines()
            labels = [list(map(float, line.split())) for line in labels]
        
        return image, labels

train_dataset = CustomYOLODataset(
    img_dir=os.path.join(download_dir, 'train/images'),
    label_dir=os.path.join(download_dir, 'train/labels'),
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss() 

In [ ]:

def train_model(model, train_loader, optimizer, criterion, num_epochs=32):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to('cuda') if torch.cuda.is_available() else images
            labels = [label.to('cuda') if torch.cuda.is_available() else label for label in labels]

            optimizer.zero_grad()

            outputs = model(images)

            loss = compute_yolo_loss(outputs, labels, criterion) 
            loss.backward()

            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

def compute_yolo_loss(outputs, labels, criterion):
    return criterion(outputs, labels)

train_model(model, train_loader, optimizer, criterion, num_epochs=32)
